In [1]:
from socket import *
import sys
# Create a server socket, bind it to a port and start listening
tcpSerSock = socket(AF_INET, SOCK_STREAM)
# Fill in start.
serverPort = 7890
tcpSerSock.bind(('',serverPort))
tcpSerSock.listen(1)
# Fill in end.
while 1:
	# Strat receiving data from the client
	print('Ready to serve...')
	tcpCliSock, addr = tcpSerSock.accept()
	print('Received a connection from:', addr)
	message = tcpCliSock.recv(4096)
	print(message)
	# Extract the filename from the given message
	print(message.split()[1])
	filename = message.split()[1].decode().partition("/")[2]
	filetouse = message.split()[1].decode().partition("//")[2].replace('/', '_')
	print('filename = ' + filename)
	fileExist = "false"
	print('filetouse = ' + filetouse)
	
	try:
		# Check wether the file exist in the cache
		f = open(filetouse[0:], "r") 
		outputdata = f.readlines() 
		print("fileexist = true")
		fileExist = "true"
		# ProxyServer finds a cache hit and generates a response message
		
		for i in range(0 , len(outputdata)):
    			tcpCliSock.send(outputdata[i].encode())
		tcpCliSock.close()
		print('Read from cache') 

		# Error handling for file not found in cache
	except IOError:
		if fileExist == "false": 
			# Create a socket on the proxyserver
			tcpCacheSock = socket(AF_INET, SOCK_STREAM)
			hostn = filename.replace("www.","",1) 
			#hostn = message.decode().split()[1].partition("//")[2].partition("/")[0]
			
			print('hostn = ' + hostn)

			try:
				# Connect to the socket to port 80
				# Fill in start.
				tcpCacheSockPort = 80
				serverName = hostn.split("/")[1]
				print((serverName, tcpCacheSockPort))
				tcpCacheSock.connect((serverName, tcpCacheSockPort))
				print('Socket connected to port 80 of the host')
				tcpCacheSock.send(message)
				print('Send the message sucessfully!')

			
				#获取服务器返回内容
				response = b''
				rec = tcpCacheSock.recv(4096)
				while rec:
					response += rec
					rec = tcpCacheSock.recv(4096)
				#print(response)
				if response.split()[1] == b'404':
							print('404')
							tcpCliSock.send("HTTP/1.1 404 Not Found\r\n\r\n".encode())
							tcpCliSock.close()
							continue

				print('response = ' + response.decode())
				tmpFile = open("./" + filetouse,"w")
				print('open the file!')
				# Fill in start.
				tmpFile.writelines(response.decode().replace('\r\n','\n'))
				tmpFile.close()
				sys.exit()
				# Fill in end.
			except:
				print("Illegal request")
		else:
			# HTTP response message for file not found
			# Fill in start.
			print('File Not Found')

			# Fill in end.
			# Close the client and the server sockets 
	tcpCliSock.close() 
			# Fill in start.
tcpSerSock.close()
			# Fill in end.


Ready to serve...
Received a connection from: ('172.24.126.100', 8787)
b'\x16\x03\x01\x02\x00\x01\x00\x01\xfc\x03\x03\x9f&\xd0s\x06\x8c<\\\x91\xa6\xd1V\xa0\xb6\x80\xc1w\xfa8\xd6\x88pt\x8cc\xfa\xc5\xe8\x0f\xc4\xf0\x1f \x8d4\xcfZ\xcc\x98\xc5\xe5&3\xb7\x04\xf3\xbcq\xa3T\xf4\xc6\xa0\x0b\xcczH\x7f\xb4\xcc\xd2\x16i\xcd\x14\x00 \x8a\x8a\x13\x01\x13\x02\x13\x03\xc0+\xc0/\xc0,\xc00\xcc\xa9\xcc\xa8\xc0\x13\xc0\x14\x00\x9c\x00\x9d\x00/\x005\x01\x00\x01\x93JJ\x00\x00\x00\x17\x00\x00\xff\x01\x00\x01\x00\x00\n\x00\n\x00\x08JJ\x00\x1d\x00\x17\x00\x18\x00\x0b\x00\x02\x01\x00\x00#\x00\x00\x00\x10\x00\x0e\x00\x0c\x02h2\x08http/1.1\x00\x05\x00\x05\x01\x00\x00\x00\x00\x00\r\x00\x12\x00\x10\x04\x03\x08\x04\x04\x01\x05\x03\x08\x05\x05\x01\x08\x06\x06\x01\x00\x12\x00\x00\x003\x00+\x00)JJ\x00\x01\x00\x00\x1d\x00 #g\x01\x06\xe3\x8cu\xf1\xe3\xa8\xa2k\x1e\xcb\xe5\x07<\x92C@\xf4\x8br\xfe\xcd"t;\x1c\xcc\xf70\x00-\x00\x02\x01\x01\x00+\x00\x07\x06\xfa\xfa\x03\x04\x03\x03\x00\x1b\x00\x03\x02\x00\x02Di\x00\x05\x00\x03

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8d in position 0: invalid start byte